1\. Write a function that converts number representation, bin<->dec<->hex. (Clearly using the corresponding python built-in functions is not fair..)

In [5]:
# Integer conversion utilities: bin <-> dec <-> hex
# Implemented without using bin(), hex(), format() or int(..., base)

_DIGITS = "0123456789ABCDEF"

def dec_to_base(n: int, base: int, prefix: bool = True) -> str:
    """Convert integer n to string in given base (2..16)."""
    if base < 2 or base > 16:
        raise ValueError("base must be between 2 and 16")
    if n == 0:
        return (f"0b0" if base == 2 and prefix else
                f"0x0" if base == 16 and prefix else
                "0")
    sign = "-" if n < 0 else ""
    n = abs(n)
    digits = []
    while n > 0:
        digits.append(_DIGITS[n % base])
        n //= base
    digits.reverse()
    body = "".join(digits)
    if prefix:
        if base == 2:
            return sign + "0b" + body
        if base == 16:
            return sign + "0x" + body
    return sign + body

def base_to_dec(s: str, base: int = None) -> int:
    """Parse string s in base (if base is None, detect 0b/0x prefixes).
       Accepts optional leading + or - and optional 0b/0x prefix."""
    if not isinstance(s, str):
        raise TypeError("input must be a string")
    s = s.strip()
    if s == "":
        raise ValueError("empty string")
    sign = 1
    if s[0] in "+-":
        if s[0] == "-":
            sign = -1
        s = s[1:].lstrip()
        if s == "":
            raise ValueError("no digits after sign")
    # detect prefix
    if s.lower().startswith("0b"):
        detected_base = 2
        s = s[2:]
    elif s.lower().startswith("0x"):
        detected_base = 16
        s = s[2:]
    else:
        detected_base = None
    if base is None:
        if detected_base is None:
            raise ValueError("base not provided and no prefix detected")
        base = detected_base
    else:
        if detected_base is not None and base != detected_base:
            # Respect explicit base parameter but allow prefix mismatch to cause error
            raise ValueError("string prefix contradicts provided base")
    if base < 2 or base > 16:
        raise ValueError("base must be between 2 and 16")
    if s == "":
        raise ValueError("no digits")
    value = 0
    for ch in s.upper():
        if ch == "_":
            continue
        if "0" <= ch <= "9":
            digit = ord(ch) - ord("0")
        elif "A" <= ch <= "F":
            digit = ord(ch) - ord("A") + 10
        else:
            raise ValueError(f"invalid digit '{ch}' for base {base}")
        if digit >= base:
            raise ValueError(f"digit '{ch}' out of range for base {base}")
        value = value * base + digit
    return sign * value

# Convenience wrappers:

def dec_to_bin(n: int, prefix: bool = True) -> str:
    return dec_to_base(n, 2, prefix=prefix)

def dec_to_hex(n: int, prefix: bool = True) -> str:
    return dec_to_base(n, 16, prefix=prefix)

def bin_to_dec(s: str) -> int:
    return base_to_dec(s, 2)

def hex_to_dec(s: str) -> int:
    return base_to_dec(s, 16)

def bin_to_hex(s: str, prefix: bool = True) -> str:
    return dec_to_hex(bin_to_dec(s), prefix=prefix)

def hex_to_bin(s: str, prefix: bool = True) -> str:
    return dec_to_bin(hex_to_dec(s), prefix=prefix)

# Example usages (for quick manual tests):
print(dec_to_bin(42))        # -> '0b101010'
print(bin_to_dec('101010'))  # -> 42
print(dec_to_hex(-255))      # -> '-0xFF'
print(hex_to_bin('0xF'))     # -> '0b1111'

0b101010
42
-0xFF
0b1111


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [30]:
def string_to_float(word):
    """Convert a 32 bit word into a single precision floating point number."""
    if len(word) != 32 or any(c not in '01' for c in word):
        raise ValueError("Input must be a 32-character string of '0' and '1'")
    sign = 1 if word[0] == '0' else -1
    exponent = bin_to_dec(word[1:9]) - 127
    mantissa = 1.0
    for i, bit in enumerate(word[9:]):
        if bit == '1':
            mantissa += 1 / (2 ** (i+1))
    return sign * mantissa * (2 ** exponent)

# Example:
print(string_to_float('01000000101000000000000000000000'))  # -> 5.0

5.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [39]:
def underflow_limit():
    x = 1.0
    while x / 2.0 > 0.0:
        x /= 2.0
    return x

def overflow_limit():
    x = 1.0
    while x * 2.0 < float('inf'):
        x *= 2.0
    return x

# Example:
print(underflow_limit())  # Smallest positive float
print(overflow_limit())   # Largest finite float

5e-324
8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [33]:
def machine_precision():
    epsilon = 1.0
    while (1.0 + epsilon / 2.0) != 1.0:
        epsilon /= 2.0
    return epsilon

# Example:
print(machine_precision())

2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [42]:
def solve_quadratic_a(a: float, b: float, c: float) -> tuple:
    """Solve quadratic equation ax^2 + bx + c = 0 with improved numerical stability."""
    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        raise ValueError("No real roots")
    sqrt_disc = discriminant**0.5
    root1 = (-b - sqrt_disc) / (2*a)
    root2 = (-b + sqrt_disc) / (2*a)
    return (root1, root2)

# Example:
print(solve_quadratic_a(0.001, 1000, 0.001))


def solve_quadratic_b(a: float, b: float, c: float) -> tuple:
    """Solve quadratic equation ax^2 + bx + c = 0 with improved numerical stability."""
    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        raise ValueError("No real roots")
    sqrt_disc = discriminant**0.5
    if b >= 0:
        root1 = (-2*c) / (b + sqrt_disc)
    else:
        root1 = (-2*c) / (b - sqrt_disc)
    root2 = c / (a * root1)
    return (root1, root2)

# Example:
print(solve_quadratic_b(0.001, 1000, 0.001))

(-999999.999999, -9.999894245993346e-07)
(-1.000000000001e-06, -999999.9999989999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
